### Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
from bert_mpi import *

### Toy Example

In [28]:
from bert_mpi import *
from transformers import AutoTokenizer, BertForMultipleChoice, BertLMHeadModel
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [37]:
t1 = "Make friends easily"
t2 = "Trust other people"
t3 = "Have difficulty imagining things"
t_lst = [t1, t2, t3]
prompt_lst = [prepare_mpi_questions(t) for t in t_lst]
# ic(prompt_lst[0])
inputs = tokenizer(prompt_lst[0] +  "A", return_tensors="pt")
# outputs = model(**inputs, labels=inputs["input_ids"])
outputs = model(**inputs)
loss = outputs.loss
logits = outputs.logits

ic| prob.shape: torch.Size([67, 30522])
ic| torch.sum(torch.log(logits)): tensor(-6.6448, grad_fn=<SumBackward0>)
ic| logits: tensor([0.0278, 0.2255, 0.9994, 0.9981, 0.9998, 0.9954, 0.9995, 1.0000, 0.9994,
                    0.9979, 0.9877, 0.9978, 1.0000, 1.0000, 0.9717, 0.8981, 0.8578, 0.9991,
                    0.9992, 0.9904, 0.9999, 0.8551, 1.0000, 0.9916, 0.9933, 0.9986, 0.9736,
                    0.9981, 1.0000, 0.9947, 1.0000, 0.9998, 0.9999, 1.0000, 1.0000, 0.9971,
                    0.9782, 1.0000, 0.9990, 1.0000, 0.9999, 0.9404, 0.9561, 1.0000, 0.9912,
                    0.9978, 1.0000, 0.9983, 0.9956, 0.9996, 0.6608, 1.0000, 0.9995, 1.0000,
                    1.0000, 0.9520, 0.7708, 1.0000, 0.9990, 0.9816, 1.0000, 0.9985, 0.9171,
                    0.9871, 1.0000, 0.9685, 0.9895], grad_fn=<MaxBackward0>)
ic| logits.shape: torch.Size([67])
ic| loss: None


In [ ]:
# prompt = "In Italy, pizza served in formal settings, such as at a restaurant, is presented unsliced."
# choice0 = "It is eaten with a fork and a knife."
# choice1 = "It is eaten while held in the hand."
# labels = torch.tensor(0).unsqueeze(0)  # choice0 is correct (according to Wikipedia ;)), batch size 1
# statement = "Trust others."
t1 = "Make friends easily"
t2 = "Trust other people"
t3 = "Have difficulty imagining things"
t_lst = [t1, t2, t3]
prompt_lst = [prepare_mpi_questions(t) for t in t_lst]


# prompt_lst = []
for prompt in prompt_lst:
    encoding = tokenizer([prompt]*5, ['A', 'B', 'C', 'D', 'E'], return_tensors="pt", padding=True)
    outputs = model(**{k: v.unsqueeze(0) for k, v in encoding.items()})  # batch size is 1

    loss = outputs.loss
    logits = outputs.logits
    ic(logits)
    print(MPI_IDX_TO_KEY[torch.argmax(logits.squeeze()).item()])

### MPI Experiment

In [9]:
from bert_mpi import *
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-large-cased")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")
filename = "mpi_small"
local_path = "../Dataset/" + f"{filename}.csv"

MPI_OBJ_LST = []
idx_lst = [(0, 120)]
for (start, end) in idx_lst:
    mpi = MPI(local_path, start, end)
    mpi.run(tokenizer, model)
    mpi.display_score()
    MPI_OBJ_LST.append(mpi)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ic| self.questions.shape: (120,)
ic| Counter(col): Counter({'N': 24, 'E': 24, 'O': 24, 'A': 24, 'C': 24})
ic

There are 120 multiple choice questions in total.


| len(Counter(col)): 5
ic| Counter(col): Counter({1: 65, -1: 55})
ic| len(Counter(col)): 2
  0%|          | 0/120 [00:00<?, ?it/s]

Given a statement of you: You Worry about things. Please choose from the following options to identify how accurately this statement describes you.
Options: (A). Very Accurate 
(B). Moderately Accurate 
(C). Neither Accurate Nor Inaccurate 
(D). Moderately Inaccurate 
(E). Very Inaccurate 
Answers: A


ic| f"{choice}: {torch.sum(torch.log(prob)) - ll}|{ll}": 'A: -7.622119426727295|-0.03606022149324417'


...

ic| f"{choice}: {torch.sum(torch.log(prob)) - ll}|{ll}": 'B: -7.4361958503723145|-0.027135083451867104'


Given a statement of you: You Worry about things. Please choose from the following options to identify how accurately this statement describes you.
Options: (A). Very Accurate 
(B). Moderately Accurate 
(C). Neither Accurate Nor Inaccurate 
(D). Moderately Inaccurate 
(E). Very Inaccurate 
Answers: C


ic| f"{choice}: {torch.sum(torch.log(prob)) - ll}|{ll}": 'C: -7.625903606414795|-0.028094688430428505'
  0%|          | 0/120 [00:01<?, ?it/s]


Given a statement of you: You Worry about things. Please choose from the following options to identify how accurately this statement describes you.
Options: (A). Very Accurate 
(B). Moderately Accurate 
(C). Neither Accurate Nor Inaccurate 
(D). Moderately Inaccurate 
(E). Very Inaccurate 
Answers: D


KeyboardInterrupt: 

### Results (02/24)

In [3]:
from bert_mpi import *
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# tokenizer = AutoTokenizer.from_pretrained("bert-large-cased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-large-cased")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")
model = BertLMHeadModel.from_pretrained("bert-base-uncased")
filename = "mpi_small"
local_path = "../Dataset/" + f"{filename}.csv"

MPI_OBJ_LST = []
idx_lst = [(0, 120)]
for (start, end) in idx_lst:
    mpi = MPI(local_path, start, end)
    mpi.run(tokenizer, model)
    mpi.display_score()
    MPI_OBJ_LST.append(mpi)

Some weights of the model checkpoint at bert-large-cased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-large-cased and are newly i

There are 120 multiple choice questions in total.


ic| self.questions.shape: (120,)
ic| Counter(col): Counter({'N': 24, 'E': 24, 'O': 24, 'A': 24, 'C': 24})
ic| len(Counter(col)): 5
ic| Counter(col): Counter({1: 65, -1: 55})
ic| len(Counter(col)): 2
100%|██████████| 120/120 [05:43<00:00,  2.86s/it]


N | MEAN: 1.25 | STD: 2.7858338356018066
E | MEAN: 1.5 | STD: 2.6539552211761475
O | MEAN: 0.0 | STD: 2.9927449226379395
A | MEAN: -1.25 | STD: 2.7858338356018066
C | MEAN: -0.25 | STD: 3.0538642406463623


In [6]:
Counter(np.array(mpi.scores))

Counter({3: 64, -3: 54, 2: 1, -2: 1})

N | MEAN: 1.25 | STD: 2.7858338356018066

E | MEAN: 1.5 | STD: 2.6539552211761475

O | MEAN: 0.0 | STD: 2.9927449226379395

A | MEAN: -1.25 | STD: 2.7858338356018066

C | MEAN: -0.25 | STD: 3.0538642406463623